In [ ]:
! git clone https://github.com/MIC-DKFZ/nnUNet.git

In [1]:
import os

In [2]:
! nvidia-smi


Thu Nov 30 15:04:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050 Ti     Off | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              N/A / ERR! |      6MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## nnUNet_raw should exists since it is on the github
### Create Manually the other two folders the results and preprocessed

In [3]:
base_dir = os.getcwd()
nnUNet_raw = os.path.join(base_dir,'nnUNet_raw')
nnUNet_preprocessed = os.path.join(base_dir,'nnUNet_preprocessed')
results_folder = os.path.join(base_dir,'nnUNet_results')

In [4]:
print(nnUNet_raw) #Check the directory address is correct

/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_raw


In [5]:
os.environ["nnUNet_raw"] = str(nnUNet_raw)
os.environ["nnUNet_preprocessed"] = str(nnUNet_preprocessed)
os.environ["nnUNet_results"] = str(results_folder)

## Checking the data and preparing for the trainning

In [6]:
! nnUNetv2_plan_and_preprocess -d 975 --verify_dataset_integrity

Fingerprint extraction...
Dataset975_BrainSegmentation
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Experiment planning...
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 69, 'patch_size': array([160, 160]), 'median_image_size_in_voxels': array([139., 147.]), 'spacing': array([0.9375, 0.9375]), 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': (2, 2, 2, 2, 2, 2), 'n_conv_per_stage_decoder': (2, 2, 2, 2, 2), 'num_pool_per_axis': [5, 5], 'pool_op_kernel_sizes': [[1, 1], [2, 2], [2, 2], [2, 2], [2, 2], [2, 2]], 'conv_kernel_sizes': [[3, 3], [3, 3], [3, 3], [3, 3], [3, 3], [3, 3]], 'unet_max_num_features': 512, 'r

## Training
- To train - nnUNet_train TRAINER_CLASS_NAME TASK_NAME_OR_ID FOLD
- To resume - nnUNet_train TRAINER_CLASS_NAME TASK_NAME_OR_ID FOLD -c (just add -c to the training command)
- TRAINER_CLASS_NAME - 2d, 3d_fullres, 3d_lowres, 3d_cascade_fullres
- Everything will be stored in the results folder

In [7]:
! nnUNetv2_train -h

usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--use_compressed] [--npz] [--c]
                      [--val] [--val_best] [--disable_checkpointing]
                      [-device DEVICE]
                      dataset_name_or_id configuration fold

positional arguments:
  dataset_name_or_id    Dataset name or ID to train with
  configuration         Configuration that should be trained
  fold                  Fold of the 5-fold cross-validation. Should be an int
                        between 0 and 4.

options:
  -h, --help            show this help message and exit
  -tr TR                [OPTIONAL] Use this flag to specify a custom trainer.
                        Default: nnUNetTrainer
  -p P                  [OPTIONAL] Use this flag to specify a custom plans
                        identifier. Default: nnUNetPlans
  -pretrained_weights PRETRAINED_WEIGHTS
                        [O

In [9]:
! nnUNetv2_train 975 2d 3 -tr nnUNetTrainer_5epochs --npz --c

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################


This is the configuration used by this training:
Configuration name: 2d
 {'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 69, 'patch_size': [160, 160], 'median_image_size_in_voxels': [139.0, 147.0], 'spacing': [0.9375, 0.9375], 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [True], 'UNet_class_name': 'PlainConvUNet', 'UNet_base_num_features': 32, 'n_conv_per_stage_encoder': [2, 2, 2, 2, 2, 2], 'n_conv_per_stage_decoder': [2, 2, 2, 2, 2], 'num_pool_per_axis': [5, 5], 'pool_op_kernel_sizes': [[1, 1], 

## Find the best configuration

In [12]:
! nnUNetv2_find_best_configuration 975 -c 2d -f 3 -tr nnUNetTrainer_5epochs


***All results:***
nnUNetTrainer_5epochs__nnUNetPlans__2d: 0.9237843734958894

*Best*: nnUNetTrainer_5epochs__nnUNetPlans__2d: 0.9237843734958894

***Determining postprocessing for best model/ensemble***
Results were improved by removing all but the largest foreground region. Mean dice before: 0.92378 after: 0.92378
Removing all but the largest component for 1 did not improve results! Dice before: 0.90996 after: 0.86224
Results were improved by removing all but the largest component for 2. Dice before: 0.93355 after: 0.93361
Removing all but the largest component for 3 did not improve results! Dice before: 0.92784 after: 0.89648

***Run inference like this:***

nnUNetv2_predict -d Dataset975_BrainSegmentation -i INPUT_FOLDER -o OUTPUT_FOLDER -f  3 -tr nnUNetTrainer_5epochs -c 2d -p nnUNetPlans

***Once inference is completed, run postprocessing like this:***

nnUNetv2_apply_postprocessing -i OUTPUT_FOLDER -o OUTPUT_FOLDER_PP -pp_pkl_file /home/edalita/Documents/MAIA/3-Semestre/MIRMISA

## Predictions

In [22]:
! nnUNetv2_predict -d Dataset975_BrainSegmentation -i /home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_raw/Dataset975_BrainSegmentation/imagesTs/ -o /home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet_raw/Dataset975_BrainSegmentation/labelsTs/ -f  3 -tr nnUNetTrainer_5epochs -c 2d -p nnUNetPlans


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 6 cases in the source folder
Traceback (most recent call last):
  File "/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/projectMisaMira/bin/nnUNetv2_predict", line 8, in <module>
    sys.exit(predict_entry_point())
  File "/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation/nnUNet/nnunetv2/inference/predict_from_raw_data.py", line 838, in predict_entry_point
    predictor.predict_from_files(args.i, args.o, save_probabilities=args.save_probabilities,
  File "/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18

In [15]:
print(base_dir)

/home/edalita/Documents/MAIA/3-Semestre/MIRMISAProject/IBSR18-tissue-segmentation


In [ ]:
! nnUNetv2_predict 